# Linked Lists

## Agenda

1. The `LinkedList` and `Node` classes  
2. Implementing `append`
3. Implementing deletion
4. Bidirectional links (Doubly-linked list) & Sentinel head
5. Incorporating a "cursor"
6. Search?
7. Run-time analysis

## 1. The `LinkedList` and `Node` classes

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next
    
    def __init__(self):
        self.head = None
        self.count = 0
    
    def prepend(self, value): # O(1)
        self.head = LinkedList.Node(value, self.head)
        self.count += 1
    
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head
        while n:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(repr(x) for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
lst

## 2. Implementing `append`

### Option 1

In [ ]:
class LinkedList (LinkedList): # note: using inheritance to extend prior definition
    def append(self, value): # O(N) !!!!!!! <-- this is BAD
        if self.head:
            # step 1: find the last node
            n = self.head
            while n.next: # traverses all nodes in the list (O(N))
                n = n.next
            
            # step 2: make the last node refer to the newly created one
            n.next = LinkedList.Node(value)
            self.count += 1
        else:
            self.prepend(value)

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

### Option 2

In [ ]:
class LinkedList (LinkedList):
    def __init__(self):
        self.head = self.tail = None
        self.count = 0
        
    def prepend(self, value): # O(1)
        self.head = LinkedList.Node(value, next=self.head)

        if self.tail is None:
            self.tail = self.head
        
        self.count += 1
        
    def append(self, value): # O(1)
        if not self.tail:
            self.prepend(value)
        else:
            self.tail.next = self.tail = LinkedList.Node(value)
            self.count += 1

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

## 3. Implementing deletion

### Deleting the head

In [ ]:
class LinkedList (LinkedList):
    def del_head(self): # O(1)
        assert len(self) > 0
        self.head = self.head.next
        self.count -= 1

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_head()
lst.del_head()
lst

### Deleting the tail

In [ ]:
class LinkedList (LinkedList):
    def del_tail(self): # O(N)
        assert len(self) > 0
        if self.head.next is None:
            # there is only one node
            self.head = self.tail = None
        else: 
            # we have at least two nodes

            # step 1: find the node prior to the tail
            n = self.head
            while n.next.next: # O(N)
                n = n.next
            
            # step 2: remove the last node and update the tail reference
            n.next = None
            self.tail = next
        
        self.count -= 1

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_tail()
lst.del_tail()
lst

## 4. Bidirectional links (Doubly-linked list) & Sentinel head

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.count = 0
        self.head = LinkedList.Node(None) # sentinel node -- not element containing!
        self.head.prior = self.head.next = self.head
        
    def prepend(self, value): # O(1)
        self.count += 1
        n = LinkedList.Node(value, prior=self.head, next=self.head.next)
        self.head.next.prior = n
        self.head.next = n
        
    def append(self, value): # O(1)
        self.count += 1
        n = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        self.head.prior.next = n
        self.head.prior = n
        
    def __getitem__(self, idx): # O(N) 
        assert idx >= 0 and idx < len(self)
        n = self.head.next
        for _ in range(idx):
            n = n.next
        return n.val
        
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
for i in range(10):
    lst.append(i)
lst

## 5. Incorporating a "cursor"

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.head = self.cursor = LinkedList.Node(None)
        self.head.prior = self.head.next = self.head
        self.count = 0
                
    def append(self, value):
        n = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        n.prior.next = n.next.prior = n
        self.count += 1

    def __getitem__(self, idx):
        assert idx >= 0 and idx < len(self)
        n = self.head.next
        for _ in range(idx):
            n = n.next
        return n.val
        
    def cursor_set(self, idx):
        assert idx >= 0 and idx < len(self)
        self.cursor = self.head.next
        for _ in range(idx):
            self.cursor = self.cursor.next
    
    def cursor_insert(self, value): # O(1)
        assert self.cursor is not self.head
        n = LinkedList.Node(value, prior=self.cursor.prior, next=self.cursor)
        n.prior.next = n.next.prior = n
        self.count += 1
    
    def cursor_delete(self): # O(1)
        assert self.cursor is not self.head
        n = self.cursor 
        self.cursor = self.cursor.next
        n.prior.next = n.next
        n.next.prior = n.prior
        self.count -= 1
        
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

In [ ]:
lst.cursor_set(4)
for x in 'abcd':
    lst.cursor_insert(x)
lst

In [ ]:
lst.cursor_set(8)
for _ in range(4):
    lst.cursor_delete()
lst

## 6. Search?

Linear search with $O(N)$ is the only option when the list is unsorted, but we previously implemented binary search, which runs in $O(\log N)$ time given a sorted list as input.

Does this extend to linked lists?

In [ ]:
def contains(lst, x):
    lo = 0
    hi = len(lst)-1
    while lo <= hi: 
        mid = (lo + hi) // 2
        if x < lst[mid]:
            hi = mid - 1
        elif x > lst[mid]:
            lo = mid + 1
        else:
            return True
    else:
        return False

In [ ]:
import timeit
import matplotlib.pyplot as plt
import numpy as np

# runtimes when searching for different values in a fixed-size list

def build_sorted_llist(n):
    lst = LinkedList()
    for x in range(n):
        lst.append(x)
    return lst
    
ts = [timeit.timeit(stmt=f'contains(lst, {x})', 
                    setup='lst = build_sorted_llist(100)',
                    globals=globals(), 
                    number=100)
      for x in range(100)]

plt.plot(range(100), ts, 'or');

In [ ]:
# runtimes when searching for an edge-value in lists of increasing size

ns = np.linspace(10, 1000, 50, dtype=int)
ts = [timeit.timeit('contains(lst, 0)', 
                    setup=f'lst=build_sorted_llist({n})',
                    globals=globals(),
                    number=1000)
      for n in ns]

plt.plot(ns, ts, 'or');

## 7. Run-time analysis

Run-time complexities for circular, doubly-linked list of $N$ elements:

- indexing (position-based access) = $O(N)$
- search (unsorted) = $O(N)$
- search (sorted) = $O(N)$ --- binary search isn't possible!
- prepend = $O(1)$
- append = $O(1)$
- insertion at arbitrary position: indexing = $O(N)$ + insertion = $O(1)$
- deletion of arbitrary element: indexing = $O(N)$ + deletion = $O(1)$